In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import wordnet as wn
nltk.download('wordnet')
import scipy

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
df = pd.read_csv('/content/drive/My Drive/simverb-3500-data/data/SimVerb-3500.txt',sep='\t',names=['verb_1','verb_2','POS','Similarity','info'])
df = df[['verb_1','verb_2','Similarity']]
df.head()

,verb_1,verb_2,Similarity
0,take,remove,6.81
1,walk,trail,4.81
2,feed,starve,1.49
3,shine,polish,7.80
4,calculate,add,5.98


In [ ]:
def fix_range(scores,low,high):
    new_scores = []
    #print(scores)
    minimum = min(scores)
    maximum = max(scores)
    dif = maximum - minimum
    ran = high - low
    for i in range(len(scores)):
        new_val = np.round(low + ((scores[i]-minimum)*ran/dif),2)
        new_scores.append(new_val)
    return new_scores

In [ ]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
word2vec_pretrained = KeyedVectors.load_word2vec_format("/content/drive/My Drive/GoogleNews-vectors-negative300.bin", binary=True)

In [ ]:
from gensim.models import KeyedVectors
filename = '/content/drive/My Drive/glove.6B.300d.txt.word2vec'
glove = KeyedVectors.load_word2vec_format(filename, binary=False)

In [ ]:
def verb_similarity(v1,v2):
    sim1 = []
    sim2 = []
    sim3 = []
    synset1 = wn.synsets(v1,pos='v')
    synset2 = wn.synsets(v2,pos='v')
    word2vec_sim = word2vec_pretrained.wv.similarity(v1,v2)
    try:
        glove_sim = glove.wv.similarity(v1,v2)
    except KeyError:
        glove_sim = 0
    w1 = synset1[0]
    w2 = synset2[0]
    wup_sim = w1.wup_similarity(w2)
    path_sim = w1.path_similarity(w2)
    lch_sim = w1.lch_similarity(w2)
    for syn1 in synset1:
          for syn2 in synset2:
              sim1.append(syn1.wup_similarity(syn2))              
              sim2.append(syn1.path_similarity(syn2))
              sim3.append(syn1.lch_similarity(syn2))          
    wup_max_sim = max(sim1)
    path_max_sim = max(sim2)
    lch_max_sim = max(sim3)
    sim_list = [wup_sim,path_sim,lch_sim,wup_max_sim,path_max_sim,lch_max_sim,word2vec_sim,glove_sim]
    return sim_list 

In [ ]:
result = [verb_similarity(x, y) for x, y in zip(df['verb_1'], df['verb_2'])]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if __name__ == '__main__':


In [ ]:
headers = ['wup_sim','path_sim','lch_sim','wup_max_sim','path_max_sim','lch_max_sim','word2vec_sim','glove_sim']

In [ ]:
result_df = pd.DataFrame(result,columns=headers)

In [ ]:
result_df

,wup_sim,path_sim,lch_sim,wup_max_sim,path_max_sim,lch_max_sim,word2vec_sim,glove_sim
0,0.400000,0.250000,1.871802,1.000000,1.000000,3.258097,0.293123,0.356702
1,0.250000,0.200000,1.648659,0.400000,0.333333,2.159484,0.246943,0.351702
2,0.222222,0.125000,1.178655,0.333333,0.200000,1.648659,0.464113,0.312260
3,0.250000,0.142857,1.312186,1.000000,1.000000,3.258097,0.437054,0.059131
4,0.222222,0.125000,1.178655,0.857143,0.500000,2.564949,0.254452,0.152029
...,...,...,...,...,...,...,...,...
3495,0.181818,0.100000,0.955511,0.400000,0.142857,1.312186,0.228921,0.122216
3496,0.166667,0.090909,0.860201,0.933333,0.500000,2.564949,0.192815,0.278167
3497,0.250000,0.142857,1.312186,0.285714,0.166667,1.466337,0.219344,0.325587
3498,0.250000,0.142857,1.312186,0.285714,0.166667,1.466337,0.105011,-0.024572


In [ ]:
human_annotation = df['Similarity'].tolist()
scaled_sim = fix_range(human_annotation,0,1)

In [ ]:
wup_list = result_df['wup_sim'].tolist()
path_list = result_df['path_sim'].tolist()
lch_list = result_df['lch_sim'].tolist()
wup_max_list = result_df['wup_max_sim'].tolist()
path_max_list = result_df['path_max_sim'].tolist()
lch_max_list = result_df['lch_max_sim'].tolist()
word2vec_list = result_df['word2vec_sim'].tolist()
glove_list = result_df['glove_sim'].tolist()

In [ ]:
wup_correlation = scipy.stats.pearsonr(scaled_sim, wup_list)
path_correlation = scipy.stats.pearsonr(scaled_sim, path_list)
lch_correlation = scipy.stats.pearsonr(scaled_sim, lch_list)
wup_max_correlation = scipy.stats.pearsonr(scaled_sim, wup_max_list)
path_max_correlation = scipy.stats.pearsonr(scaled_sim, path_max_list)
lch_max_correlation = scipy.stats.pearsonr(scaled_sim, lch_max_list)
word2vec_correlation = scipy.stats.pearsonr(scaled_sim,word2vec_list)
glove_correlation = scipy.stats.pearsonr(scaled_sim,glove_list)

In [ ]:
print("wup correlation %.2f" % wup_correlation[0], "\npath_correlation %.2f " % path_correlation[0],
      "\nlch correlation %.2f" % lch_correlation[0], "\nwup_max_correaltion %.2f " % wup_max_correlation[0],
      "\npath_max_correlation %.2f " % path_max_correlation[0], "\nlch_max_correlation %.2f" % lch_max_correlation[0],
      "\nword2vec_correlation %.2f" % word2vec_correlation[0], "\nglove_correlation %.2f" % glove_correlation[0])

wup correlation 0.28 
path_correlation 0.28  
lch correlation 0.23 
wup_max_correaltion 0.48  
path_max_correlation 0.46  
lch_max_correlation 0.46 
word2vec_correlation 0.38 
glove_correlation 0.23


In [ ]:
df['scaled_sim'] = scaled_sim
verb_similarity_comp = pd.concat([df, result_df], axis=1, sort=False)

In [ ]:
verb_similarity_comp

,verb_1,verb_2,Similarity,scaled_sim,wup_sim,path_sim,lch_sim,wup_max_sim,path_max_sim,lch_max_sim,word2vec_sim,glove_sim
0,take,remove,6.81,0.68,0.400000,0.250000,1.871802,1.000000,1.000000,3.258097,0.293123,0.356702
1,walk,trail,4.81,0.48,0.250000,0.200000,1.648659,0.400000,0.333333,2.159484,0.246943,0.351702
2,feed,starve,1.49,0.15,0.222222,0.125000,1.178655,0.333333,0.200000,1.648659,0.464113,0.312260
3,shine,polish,7.80,0.78,0.250000,0.142857,1.312186,1.000000,1.000000,3.258097,0.437054,0.059131
4,calculate,add,5.98,0.60,0.222222,0.125000,1.178655,0.857143,0.500000,2.564949,0.254452,0.152029
...,...,...,...,...,...,...,...,...,...,...,...,...
3495,impose,cheat,1.16,0.12,0.181818,0.100000,0.955511,0.400000,0.142857,1.312186,0.228921,0.122216
3496,rebel,protest,7.64,0.77,0.166667,0.090909,0.860201,0.933333,0.500000,2.564949,0.192815,0.278167
3497,collaborate,conspire,4.23,0.42,0.250000,0.142857,1.312186,0.285714,0.166667,1.466337,0.219344,0.325587
3498,conspire,protest,1.83,0.18,0.250000,0.142857,1.312186,0.285714,0.166667,1.466337,0.105011,-0.024572


In [ ]:
word1 = 'Integrate'
word2 = 'arrange'

In [ ]:
synset1 = wn.synsets(word1,pos='v')
synset2 = wn.synsets(word2,pos='v')

In [ ]:
sim = []
for syn1 in synset1:
    for syn2 in synset2:
        sim.append(syn1.wup_similarity(syn2))                       
    wup_max_sim = max(sim)

In [ ]:
wup_max_sim

0.4